# Description

Score response texts for 'genericness' based on semantic similarity to a reference set of generic sentences


### Environment details:
   - server: gimli
   - conda env: readvisor38

In [2]:
from pathlib import Path
from typing import Dict, List
from collections import Counter

from multiprocessing_utils import parallelize_dataframe

import numpy as np
import pandas as pd
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None) # don't truncate cell contents

import torch
from torch import Tensor
from tqdm.notebook import tqdm
tqdm.pandas()
import numpy as np
import matplotlib.pyplot as plt

import spacy

# from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = spacy.load('/srv/scratch2/kew/spacy_models/en_core_web_sm-2.3.1/en_core_web_sm/en_core_web_sm-2.3.1', disable=["tagger", "parser", "ner"])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
# import qgrid


from sentence_transformers import SentenceTransformer, util

/mnt/storage/clwork/users/kew/INSTALLS/anaconda3/envs/respondelligent/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
test = False

In [ ]:
RANDOM_SEED = 42
st_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
st_model.max_seq_length = 256

In [7]:
df = pd.read_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.pkl')

if test:
    df = df[df['split'] == 'test']
    print(len(df))
    
print(df.columns)

Index(['Unnamed: 0', 'domain', 'rating', 'review_author', 'response_author',
       'review_clean', 'response_clean', 'sentiment', 'db_internal_id',
       'establishment', 'trip_id', 'trip_url', 'country', 'split',
       'score:review_response_length_ratio', 'score:response_sentence_length',
       'score:genericness_semantic_avg', 'score:genericness_length_ratio',
       'score:review_response_wmd', 'rrgen_id'],
      dtype='object')


In [5]:
# from spacy.tokenizer import Tokenizer
# from spacy.lang.en import English
# # nlp = English()
# # # Create a blank Tokenizer with just the English vocab
# # tokenizer = Tokenizer(nlp.vocab)
# # tokenizer.add_pipe(nlp.create_pipe('sentencizer'))
# nlp = spacy.load('/srv/scratch2/kew/spacy_models/en_core_web_sm-2.3.1/en_core_web_sm/en_core_web_sm-2.3.1')


In [6]:
# df = pd.read_csv('/srv/scratch6/kew/bart/hospo_respo/en/500k/baseline/inference/testset_entropy_scores.tsv', sep='\t', header=0, index_col=0)

In [8]:
gen_output_file = f'/home/user/kew/readvisor_proj/RESPONSE_GENERATION/hospo_respo_datasets/generic_rrgen_outputs/en.txt'

# collect all 'generic' sentences a model has generated more than once
gen_outputs = Counter()
with open(gen_output_file, 'r', encoding='utf8') as f:
    for line in f:
        c, sent = line.strip().split('\t')
        if int(c) > 1:
            gen_outputs[sent] = int(c)

print(len(gen_outputs))
print()
print('most common output sentences:')
gen_sents_sorted_sorted = gen_outputs.most_common()
for sent, count in gen_sents_sorted_sorted[:10]:
    print(f'{(count/sum(gen_outputs.values()))*100:.5f}%\t{sent}')

print()
print('least common output sentences:')
for sent, count in gen_sents_sorted_sorted[-10:]:
    print(f'{(count/sum(gen_outputs.values()))*100:.5f}%\t{sent}')

9018

most common output sentences:
12.30615%	thank you for taking the time to share your experience with us and the <NAME> community
4.28591%	thank you for taking the time to share your experience with us
4.11230%	thank you for taking the time to review your recent stay with us
3.96536%	thank you for taking the time to leave us a review
3.95961%	thank you for taking the time to write a review
3.03976%	<NAME> , we do look forward to welcoming you back in the near future
2.95273%	we are delighted to read you enjoyed our <DIGIT> star standards in customer service and quality accommodation at meriton suites herschel street , <LOC> and we do hope you were able to take some time to relax and unwind in our resort style leisure facilities throughout your stay
2.76776%	we are delighted to hear that you enjoyed your experience with us and we look forward to welcoming you back again soon
2.72065%	i am sorry to hear that you did not enjoy your stay with us
2.56033%	it is our pleasure to welcome y

In [9]:
len(gen_outputs)

9018

In [11]:
# load encoded generic sentences (if they already exist)
gen_sents = [sent for sent, _ in gen_outputs.items()]
print(len(gen_sents))
gen_embs = st_model.encode(gen_sents, batch_size=128, show_progress_bar=True, convert_to_tensor=True)
print(gen_embs.shape)
# torch.save(gen_embs, '/home/user/kew/readvisor_proj/RESPONSE_GENERATION/hospo_respo_datasets/generic_rrgen_outputs/en.paraphrase-MiniLM-L6-v2.210706.pt')


9018



torch.Size([9018, 384])


In [12]:
def average_wordpiece_count(df, model, sample_fraction=0.10):
    
    if sample_fraction > 0.0:
        df = df.sample(frac=sample_fraction)
    
    df['review_wps'] = df['review_clean'].progress_apply(lambda x: len(model.tokenizer.tokenize(x)))
    print(df['review_wps'].describe())
    
    df['response_wps'] = df['response_clean'].progress_apply(lambda x: len(model.tokenizer.tokenize(x))) 
    print(df['response_wps'].describe())
    
    return

# average_wordpiece_count(df, st_model)

def reduce_gen_sent_emb_matrix(gen_embs, gen_sents, sim_threshold=0.95):
    # find similar sentences in store of generic sentences
    seen = set()
    discard = set()

    for i in tqdm(range(len(gen_embs))):
        if i not in discard:
            seen.add(i)
            emb_i = gen_embs[i].unsqueeze(0)
            cosine_scores = util.pytorch_cos_sim(emb_i, gen_embs).squeeze(0)

            x = (cosine_scores > sim_threshold).nonzero(as_tuple=False).T.squeeze(0).tolist()
            discard.update(x)
    
        else:
            pass

    print(len(seen))
    
    gen_sents_reduced = [gen_sents[i] for i in enumerate(gen_sents) if i in seen]
    print(len(gen_sents_reduced))
    
    # get identity vector containing indices of 'base' sentence embeddings
    seen_tensor = torch.LongTensor(list(seen))
    gen_embs_reduced = torch.index_select(gen_embs, 0, seen_tensor)
    print(gen_embs_reduced.shape)

    return gen_embs_reduced, gen_sents_reduced

# gen_embs, gen_sents = reduce_gen_sent_emb_matrix(gen_embs, gen_sents, sim_threshold=0.95)

In [13]:
def normalise_sents(text):
    sents = [str(s).lower() for s in nlp(text).sents]
    return sents

def max_pool_semantic_similarity(sents: List[str], gen_sent_embs: Tensor, verbose: bool = False):
    """
    calculates sentence-average semantic similarity to all generic_sent_embs
    """
    embeddings = st_model.encode(sents, convert_to_tensor=True)
    
    # Compute cosine-similarities for each sentence with each other sentence
    cosine_scores = util.pytorch_cos_sim(embeddings, gen_sent_embs)
    max_pool = torch.max(cosine_scores, dim=-1, keepdim=False)    
        
    if verbose:
        for i in range(len(sents)):
            gen_sent_idx = max_pool.indices[i]
            print(f'{max_pool.values[i]}: {sents[i]} ~~ {gen_sents[gen_sent_idx]}')
 
    mean = torch.mean(max_pool.values)
            
    return mean.item()
        
    
def calc_generic_avg(text: str, gen_sent_embs: Tensor = gen_embs, verbose: int = 0) -> float:
    """
    returns semantic similarity with the embedded generic sentences calculated by averaging across sentences.
    NB. identifies generic texts based on semantic textual similarity (cosine) with sentence transformers.
    NB. lower score is better!
    """
    
    sents = normalise_sents(text)
    if verbose:
        print([sent for sent in sents])
        
    if sents:
        sem_sim_mean = max_pool_semantic_similarity(sents, gen_sent_embs, verbose=verbose)
#         print(sem_sim_mean)
    else:
        sem_sim_mean = np.inf # positive infinity if response is empty
    
    if verbose:
        print(f'sent sim mean: {sem_sim_mean} - {text[:180]}...')

    return sem_sim_mean

In [14]:
## testing
text = df.iloc[24646]['response_clean']
print(text)
calc_generic_avg(text, gen_sent_embs=gen_embs, verbose=True)

Thank you for the 5 star review love2walkintherain! The friendly team at Seawall work hard to make sure our guests have a great stay from check in to check out, and I'm glad we were able to help you by finding an apartment appropriate for your work. Hopefully we will see you at Seawall again next time!
['thank you for the 5 star review love2walkintherain!', "the friendly team at seawall work hard to make sure our guests have a great stay from check in to check out, and i'm glad we were able to help you by finding an apartment appropriate for your work.", 'hopefully we will see you at seawall again next time!']
0.6254612803459167: thank you for the 5 star review love2walkintherain! ~~ thank you for your lovely review
0.6461806893348694: the friendly team at seawall work hard to make sure our guests have a great stay from check in to check out, and i'm glad we were able to help you by finding an apartment appropriate for your work. ~~ we are pleased to hear you enjoyed our convenient loc

0.6399007439613342

In [15]:
def run_scorer_in_parallel(df):
    df['score:genericness_sent_level_sts_avg'] = df['response_clean'].progress_apply(lambda x: calc_generic_avg(x))
    return df

df = parallelize_dataframe(df, run_scorer_in_parallel, n_cores=28)


Running jobs on 28 CPU(s)


  0%|          | 0/28 [00:00<?, ?it/s]

100%|██████████| 28/28 [1:28:16<00:00, 189.14s/it]   


Time taken: 5298.96 seconds


In [16]:
df.sort_values(by=['score:genericness_sent_level_sts_avg'], ascending=True)

,Unnamed: 0,domain,rating,review_author,response_author,review_clean,response_clean,sentiment,db_internal_id,establishment,trip_id,trip_url,country,split,score:review_response_length_ratio,score:response_sentence_length,score:genericness_semantic_avg,score:genericness_length_ratio,score:review_response_wmd,rrgen_id,score:genericness_sent_level_sts_avg
217428,2909273,Hotel,1,Cassiexx,DorothyBristol,"Wrong ---SEP--- ME and my friend went to stay at this hotel, the decoration was very old but that didnt bother us as we were out most of the time. The barman came into our room and tried to have sex with me and touch me up, then tried it with my friend..we phoned the police and reported him and he is now on unconditional bail being charged of sexual harrasement. I would never reccommend that a young group or couple should stay here. The only good thing was the pool and the breakfast was very stingy!!",The member of staff who was accused by the young girls was not prosecuted. The case was dropped.,2 -2 2,459998,Laguna Hotel,585880,https://www.tripadvisor.co.uk/Hotel_Review-g186262-or495-d585880-Reviews-Laguna_Hotel-Bournemouth_Dorset_England.html,UK,train,2.000000,2,0.282389,0.0,5.159015,2909273,0.301155
116174,2720165,Hotel,1,Adam G,stevejohnsonwork,"Smells like OAPS stale urine ---SEP--- what can I say that you don't already know, stay here and you will catch either, aids, herpes or the plague. Rooms are small, showers don't work, staff rob from you, great location though but i'd rather stay in North Korea and commute",The basis of this review is untrue and inflammatory. There is no record police or otherwise of this person being burgled whilst staying here and factually you do not get any of the diseases he has described from a hostel bedroom.,2 -1 3,904771,The Grapevine Seafront,1090786,https://www.tripadvisor.co.uk/Hotel_Review-g186273-or155-d1090786-Reviews-The_Grapevine_Seafront-Brighton_East_Sussex_England.html,UK,train,1.000000,2,0.334252,0.0,4.576758,2720165,0.301393
220356,1481991,Hotel,5,curtida1,GVGuestHouse,"Whittier Suite II ---SEP--- My wife and I stayed at the Whittier Suite II for our first anniversary. The entire stay was amazing. From fresh strawberries and champaign upon arrival to all kinds of snacks and breakfast good available in the kitchen. I was in Columbus for a meeting on the Friday afternoon of our reservation. My meeting ended two hours early and the guest house allowed me to check in extremely early (@2:00) instead of finding somewhere to stay busy till 4:00 (Thanks a million!!). I love not having to check in at a desk...the electronic check in was very convenient and thee were three our four amazing/elegant restaurants within walking distance. **I was a little disappointed with the tax associated with this bed and breakfast. The room itself was listed as $275 but I ended up paying a 10% room tax as well as the normal 6.5% sales tax. While this might have been mentioned in the fine print, I was a little taken by surprise. While the room and all the extras was DEFINATELY worth the $321 we spent after taxes and extras, It would have been nice if the GVGH would point out this extra fee/tax. Overall; however, Great experience. I'll definitely be back.","Every major metro area in the country assesses a bed tax. Franklin County Ohio levies a 10% 'bed' tax on every hotel room sold, in addition to the State of Ohio sales tax of 6.75%. The 'bed tax' in Chicago is 16.75%, in Cleveland it is 15.25%, and Toledo and Cincinnati Ohio are both at 10.5% making the Columbus rate seem reasonable by comparison. The German Village Guest House is completely transparent to our Guests about this pass-through tax that all hotels must share.",2 -2 2,400700,German Village Guest House,634792,https://www.tripadvisor.com/Hotel_Review-g50226-or425-d634792-Reviews-German_Village_Guest_House-Columbus_Ohio.html,US,train,3.250000,4,0.321740,0.0,4.767977,1481991,0.302143
288583,239833,Hotel,1,MNsota,VOSullivan,"A dump! ---SEP--- Terrible

In [17]:
# save newly computed scores
# df['score:genericness_sent_level_sts_avg'].to_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.gen_sts_scores.pkl')


In [18]:
# new_col = pd.read_pickle('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/500k/trip_hotels.gen_sts_scores.pkl')
# new_col == df['score:genericness_sent_level_sts_avg']

0         True
1         True
2         True
3         True
4         True
          ... 
499995    True
499996    True
499997    True
499998    True
499999    True
Name: score:genericness_sent_level_sts_avg, Length: 500000, dtype: bool

In [10]:
## Sanity checks
# t = "Many thanks for your review following your recent stay with us. In particular a big thank you for your glowing compliments regarding our staff who we are enormously proud of and do so much to ensure that our guests enjoy staying at The Royal We look forward to seeing you again soon. Mike MacDermid General Manager"
# sents = [str(s).lower() for s in nlp(t).sents]
# embs = st_model.encode(sents, convert_to_tensor=True)
    
# # Compute cosine-similarities for each sentence with each other sentence
# cosine_scores = util.pytorch_cos_sim(embs, gen_embs)
# print(cosine_scores.shape)
# max_pool = torch.max(cosine_scores, dim=-1, keepdim=False)    
# print(max_pool)

# for i in range(len(sents)):
#     gen_sent_idx = max_pool.indices[i]
#     print(f'{max_pool.values[i]}: {sents[i]} ~~ {gen_sents[gen_sent_idx]}')

# mean = torch.mean(max_pool.values)
# print(mean.item())

torch.Size([4, 12385])
torch.return_types.max(
values=tensor([0.8938, 0.7155, 0.9899, 0.6420]),
indices=tensor([   24, 11586,    19,  7597]))
0.8937948346138: many thanks for your review following your recent stay with us. ~~ thank you for taking the time to post your review following your recent stay with us
0.7155090570449829: in particular a big thank you for your glowing compliments regarding our staff who we are enormously proud of and do so much to ensure that our guests enjoy staying at the royal ~~ we are delighted to hear that you enjoyed your stay with us and we look forward to welcoming you back to the royal in the future
0.9899359941482544: we look forward to seeing you again soon. ~~ we look forward to seeing you again soon
0.6420048475265503: mike macdermid general manager ~~ sincerely , <NAME> general manager
0.8103111982345581


In [34]:
# # print(df['score:review_response_length_ratio'].describe())
# # print()
# # print(df['score:response_sentence_length'].describe())
# # print()
# # print(df['score:genericness_semantic_avg'].describe())
# # print()
# # print(df['score:genericness_length_ratio'].describe())

# def simple_token_length_calc(df):    
#     df['src_tok_len'] = df.progress_apply(lambda x: len(x.review_clean), axis=1)
#     df['tgt_tok_len'] = df.progress_apply(lambda x: len(x.response_clean), axis=1)
#     return df
# # df = simple_token_length_calc(df)

# def simple_char_length_calc(df):    
#     df['src_len'] = df.review_clean.str.len()
#     df['tgt_len'] = df.response_clean.str.len()
#     df['src_tgt_len_ratio'] = df['src_tgt_len'] / df['tgt_tok_len']
#     print(df['src_tgt_len_ratio'].describe())
#     df['src_tgt_len_ratio'].plot()
#     return df

# simple_char_length_calc(df)

# len(df[df['src_tgt_len_ratio'] <= 2.5])

# pd.set_option('display.max_colwidth', None)
# df[df["src_tgt_len_ratio"] > 2.5][['review_clean', 'response_clean']]

# df['src_tgt_len'] = df['src_tok_len'].astype(int)
# df['tgt_tok_len'] = df['tgt_tok_len'].astype(int)
# print(df['src_tok_len'].describe())
# print(df['tgt_tok_len'].describe())

# df['src_tgt_len_ratio'] = df['src_tgt_len'] / df['tgt_tok_len']
# print(df['src_tgt_len_ratio'].describe())


# # length filtering
# print('REMOVING EMPTY STRING VALUES FROM DF WITH LENGTH:', len(df))
# df = df[(df['review_clean'] != '') & (df['response_clean'] != '')]
# print('REMOVED ITEMS DF LENGTH:', len(df))

# df['src_tgt_len_ratio'].plot()

# df['src_tok_len'].plot()

In [ ]:
# col_name_outfile_mapping = {
#     'rrgen_id': 'rrgen_id', 
#     'review_clean': 'review', # normal review 
#     'response_clean': 'response',  # normal response
#     'rating': 'rating', # normal review rating
#     'establishment': 'establishment',
# }

# def write_file(series, outfile):
#     with open(outfile, 'w', encoding='utf8') as f:
#         for line in series.to_list():
#             f.write(f'{line}\n')
#     return

# def generate_fairseq_input_files(df,
#                                  outdir: str,
#                                  col_name_outfile_mapping: Dict,
#                                  split_col: str,
#                                  n: int = 0):
#     """
#     Generates multiple individual files (one per column).
#     For each split (train/test/valid) lines in each output file must correspond with each other!
#     """
#     for split in df[split_col].unique():  
        
#         split_df = df[df[split_col] == split]
        
#         # shuffle train set - mainly required after upsampling!
#         if split == 'train':
#             split_df = split_df.sample(frac=1, random_state=RANDOM_SEED)
        
#         if n: # just take a head of dataframe
#             if split == 'train':
#                 split_df = split_df.head(n)
#             else:
#                 split_df = split_df.head(int(n*0.1))

#         print(f'{split} split has length: {len(split_df)}')

#         for k, v in col_name_outfile_mapping.items():
#             write_file(split_df[k], outdir / f'{split}.{v}')
        
#     print('Done!')
#     return

# outdir = Path('/srv/scratch6/kew/bart/hospo_respo/en/data/hotel/unfiltered/')
# generate_fairseq_input_files(htl, outdir, col_name_outfile_mapping, 'split')